layer 2개

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
(train_input, train_target) , (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [3]:
# train과 valid 데이터 분리
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28) # 2차원 배열로 변환하는것은 현업, CNN에서는 하지 않는다. 배우는 과정이라서 하는것

train_scaled , val_scaled, train_target, val_target = train_test_split(
    train_scaled , train_target , test_size= 0.2, random_state=42
)


> 딥러닝은 모델을 만들기 전 설계서를 만들어 보면서 해야한다

### Layer를 추가하는 방법 - 1 

In [4]:
# Layer 만들기
dense1 = keras.layers.Dense(100 , activation = 'sigmoid', input_shape = (784,)) # 은닉층
dense2 = keras.layers.Dense(10 , activation = 'softmax') # 출력층

In [5]:
# 전체 모델에 층을 추가
model = keras.Sequential([dense1 , dense2])

2022-07-20 14:36:00.415283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Summary (Layer 확인)
model.summary() 
# Param : 가중치 갯수 
# 784*100 (w) , 100 (b) = 78500 
# Param갯수가 줄어 나가야 한다.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Layer를 추가하는 방법 - 2

In [7]:
model = keras.Sequential(
    [
        keras.layers.Dense(100, activation = "sigmoid" , input_shape = (784,) , name = 'hidden'),
        keras.layers.Dense(10, activation = "softmax" , name = 'output')
    ], name = "패션 Mnist모델"
)

In [8]:
model.summary()

Model: "패션 Mnist모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Layer를 추가하는 방법 - 3
주로 사용하는 방법      
데이터마다 사용할 레이어 갯수가 다를 수 있기 때문에 if문으로 add를 조절할 수 있다. (-> 알파고)      

In [10]:
model = keras.Sequential()
model.add(keras.layers.Dense(100 , activation = "sigmoid" , input_shape = (784,)))
model.add(keras.layers.Dense(10 , activation = "softmax"))

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### 모델 훈련

In [12]:
model.compile(loss='sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled , train_target , epochs= 5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5654 - accuracy: 0.8081
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4069 - accuracy: 0.8526
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3720 - accuracy: 0.8652
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3499 - accuracy: 0.8737
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3320 - accuracy: 0.8794


### Sigmoid / Relu
- Sigmoid 함수는 숫자가 무한대로 커지게 되면 데이터의 차이가 거의 없다는 단점이 있다.
- 이를 해결한 것이 Relu 함수 인데 인공신경망에 기여한 바가 크며 음수는 모든 0으로 양수는 항상 최대값을 출력하는 함수 

sigmoid는 0.5기준으로는 구분이 잘 되지만 숫자가 커지는 경우가 애매해진다. 0.999999999 0.999999999999999 등 구분이 안 된다.        
relu는 음수는 없고 양수로 계속 커지는 함수이므로 sigmoid의 단점을 해결해준다.    

### 입력층을 단순화

In [13]:
# 입력층을 단순화 하게 
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28,28))) # 입력층을 2차원 원래대로 사용한다. -> 알아서 펼쳐줌, 제일 편함 
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Optimizer(최적화 알고리즘)
- 훈련 시에 Optimizer를 사용하여 최적화 알고리즘을 구성한다.
- 보통 사용하는 것이 확률적 경사 하강법 optimizer이다. (running rate, Jump를 얼마나 할까)


#### 적용 방법 (확률적 경사 하강법)

In [15]:
# 적용 방법 (확률적 경사 하강법) - 1
model.compile(optimizer ='sgd' , loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

In [16]:
# 적용 방법 (확률적 경사 하강법) - 2
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

In [17]:
# 적용 방법 (확률적 경사 하강법) + 학습률 변경 - 3 : sgd의 기본값 learning_rate = 0.01
sgd = keras.optimizers.SGD(learning_rate= 0.1) # 기본값보다 10배정도 더 빠르게 움직일거야
model.compile(optimizer=sgd, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

In [18]:
# 적용방법 (Momentum)
sgd = keras.optimizers.SGD(momentum= 0.9 , nesterov= True) # 기본값 0.9
model.compile(optimizer=sgd, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
# 아담에 모멘텀 방식이 적용되어 있다. 

In [19]:
# 적용방법 (Adagrad)
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

In [20]:
# 적용방법 (RMSprop)
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

In [22]:
# 적용 방법
model.compile(optimizer ='rmsprop' , loss = 'sparse_categorical_crossentropy', metrics = 'accuracy') # 위의 방법도 있지만 그냥 이렇게 쓰는게 편함

---